In [39]:
# Install requirements
!pip install langchain
!pip install google-generativeai
!pip install langchain-google-genai
!pip install langchain-community
!pip install langchain-core
!pip install langchain-text-splitters
!pip install langchain-chroma
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [40]:
# importing requirements
import pandas as pd
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
##ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
# Creating embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_chroma import Chroma

# 1. Data Gatthering
# Reading and Extracting Text from a PDF

In [18]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/DC Motors.pdf")
data = loader.load()

## 2. Creating Document chunks and loading Embedding model

In [21]:
# Gemini API for google embedding model
import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [25]:
# Creating Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.split_documents(data)

In [23]:
# loading googlr embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

## 3.Set Up vector DB  and Store Embeddings

In [47]:
vector_db = Chroma.from_documents(text_chunks, embeddings)

In [48]:
# create a retriever object
retriever = VectorStoreRetriever(vectorstore=vector_db)

## 4. Creating a prompt template

In [57]:
# prompt template
template ="""
 You are a Smart Bot that can answer questions from the context provided.

I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}

Instructions:
Provide a clear, concise, and accurate response to the question based on the given context. If needed, extract information from external sources such as the vector database or generative model. Ensure the response is relevant to the user's query, and include detailed explanations or examples if necessary.

Format:
Respond in the following format:
1. **Answer**: [Provide a direct answer to the question.]
2. **Explanation**: [Offer additional information or reasoning behind the answer, if relevant.]
3. **References**: [Cite the sources or data used, if applicable.]



 """

In [58]:
prompt = ChatPromptTemplate.from_template(template)

## 5. Query the PDF and Retrieve Relevant Documents from ChromaDB

In [55]:
# Initialize LLM
llm_pipeline = ChatGoogleGenerativeAI(
        model = "gemini-pro",
        temperature = 0.1

    )

In [59]:
## Create Stuff Docment Chain
document_chain=create_stuff_documents_chain(llm_pipeline,prompt)

In [60]:
# create a retrival chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [61]:
query1 = "What is DC Motor"
response=retrieval_chain.invoke({"input":query1})
response['answer']

"1. **Answer**: A DC motor is a machine that converts direct current (DC) power into mechanical power.\n2. **Explanation**: DC motors operate on the principle that when a current-carrying conductor is placed in a magnetic field, the conductor experiences a mechanical force. This force is used to rotate the motor's shaft, which in turn produces mechanical power.\n3. **References**:\n   - [DC Motors](https://www.electronics-tutorials.ws/dccircuits/dc-motors.html)\n   - [How DC Motors Work](https://www.allaboutcircuits.com/textbook/direct-current/chpt-10/how-dc-motors-work/)"

In [63]:
questions = ["Explain the principle of operation of a DC motor.",
             "What are the different types of DC motors and their applications?",
             "How does the speed of a DC motor vary with the load?",
             "What is the significance of back EMF in a DC motor?",
             "What are the advantages and disadvantages of DC motors compared to AC motors?"]

In [64]:
for question in questions:
  response=retrieval_chain.invoke({"input":question})
  print(response['answer'])
  print("\n")


1. **Answer**: A DC motor operates on the principle that when a current-carrying conductor is placed in a magnetic field, it experiences a mechanical force. This force is determined by Fleming's left-hand rule and is proportional to the current, magnetic field strength, and length of the conductor.
2. **Explanation**: The interaction between the magnetic field generated by the stator and the current flowing through the armature conductors creates a torque that causes the motor to rotate. The direction of rotation is determined by the polarity of the magnetic fields and the direction of current flow.
3. **References**:
   - [DC Motor Principle](https://www.electronics-tutorials.ws/electromagnetism/dc-motor-principle.html)
   - [Fleming's Left-Hand Rule](https://www.electronics-tutorials.ws/electromagnetism/fleming-left-hand-rule.html)


1. **Answer**: There are three main types of DC motors: series-wound, shunt-wound, and compound-wound.
2. **Explanation**:
   - **Series-wound motors** 